Code is taken from https://www.kaggle.com/voglinio/siamese-two-pretrained-weights-0-855

In [1]:
import gzip
# Read or generate p2h, a dictionary of image name to image id (picture to hash)
import pickle
import platform
import random
# Suppress annoying stderr output when importing keras.
import sys
from lap import lapjv
from math import sqrt
# Determine the size of each image
from os.path import isfile

import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as pil_image
from imagehash import phash
from keras import backend as K
from keras import regularizers
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, \
    Lambda, MaxPooling2D, Reshape
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import Sequence
from pandas import read_csv
from scipy.ndimage import affine_transform
from tqdm import tqdm_notebook as tqdm
import time

Using TensorFlow backend.


In [2]:
TRAIN_DF = '../data/train.csv'
SUB_DF = '../data/sample_submission.csv'
TRAIN = '../data/train/'
TEST = '../data/test/'
P2H = '../data/metadata/p2h.pickle'
P2SIZE = '../data/metadata/p2size.pickle'
BB_DF = "../data/metadata/bounding_boxes.csv"
tagged = dict([(p, w) for _, p, w in read_csv(TRAIN_DF).to_records()])
submit = [p for _, p, _ in read_csv(SUB_DF).to_records()]
join = list(tagged.keys()) + submit

In [3]:
def expand_path(p):
    if isfile(TRAIN + p):
        return TRAIN + p
    if isfile(TEST + p):
        return TEST + p
    return p

In [4]:
### LOADING
if isfile(P2SIZE):
    print("P2SIZE exists.")
    with open(P2SIZE, 'rb') as f:
        p2size = pickle.load(f)

P2SIZE exists.


In [5]:
# ### COMPARE PICKLES
# value = { k : p2size_pickle[k] for k in set(p2size_pickle) - set(p2size) }
# value

In [6]:
def match(h1, h2):
    for p1 in h2ps[h1]:
        for p2 in h2ps[h2]:
            i1 = pil_image.open(expand_path(p1))
            i2 = pil_image.open(expand_path(p2))
            if i1.mode != i2.mode or i1.size != i2.size: return False
            a1 = np.array(i1)
            a1 = a1 - a1.mean()
            a1 = a1 / sqrt((a1 ** 2).mean())
            a2 = np.array(i2)
            a2 = a2 - a2.mean()
            a2 = a2 / sqrt((a2 ** 2).mean())
            a = ((a1 - a2) ** 2).mean()
            if a > 0.1: return False
    return True

In [7]:
### LOADING
if isfile(P2H):
    print("P2H exists.")
    with open(P2H, 'rb') as f:
        p2h = pickle.load(f)

P2H exists.


In [8]:
# ### COMPARE PICKLES
# value = { k : p2h_pickle[k] for k in set(p2h_pickle) - set(p2h) }
# value

In [9]:
# For each image id, determine the list of pictures
h2ps = {}
for p, h in p2h.items():
    if h not in h2ps: h2ps[h] = []
    if p not in h2ps[h]: h2ps[h].append(p)

In [10]:
### MULTIPLE PICTURE ITEMS IN H2PS

for h, ps in h2ps.items():
    if len(ps) > 1:
        print(h, ps)

eecad0b52d4ac2f0 ['01f66ca26.jpg', 'd37179fd1.jpg']
afdac0b52a5a82b5 ['579886448.jpg', 'f50529c53.jpg']
94216bb289ccd63f ['60a3f2422.jpg', '7f7a63b8a.jpg']
ad4ac2b43d0fcaf0 ['b95d73a55.jpg', 'fb3879dc7.jpg']


In [11]:
def show_whale(imgs, per_row=2):
    n = len(imgs)
    rows = (n + per_row - 1) // per_row
    cols = min(per_row, n)
    fig, axes = plt.subplots(rows, cols, figsize=(24 // per_row * cols, 24 // per_row * rows))
    for ax in axes.flatten(): ax.axis('off')
    for i, (img, ax) in enumerate(zip(imgs, axes.flatten())): ax.imshow(img.convert('RGB'))
        

def read_raw_image(p):
    img = pil_image.open(expand_path(p))
    return img

In [12]:
# For each images id, select the prefered image
def prefer(ps):
    if len(ps) == 1: return ps[0]
    best_p = ps[0]
    best_s = p2size[best_p]
    for i in range(1, len(ps)):
        p = ps[i]
        s = p2size[p]
        if s[0] * s[1] > best_s[0] * best_s[1]:  # Select the image with highest resolution
            best_p = p
            best_s = s
    return best_p

In [23]:
h2p = {}
for h, ps in h2ps.items():
    h2p[h] = prefer(ps)

In [24]:
len(h2p), list(h2p.items())[:5]

(33317,
 [('d26698c3271c757c', '0000e88ab.jpg'),
  ('ba8cc231ad489b77', '0001f9222.jpg'),
  ('bbcad234a52d0f0b', '00029d126.jpg'),
  ('c09ae7dc09f33a29', '00050a15a.jpg'),
  ('d02f65ba9f74a08a', '0005c1ef8.jpg')])

In [25]:
# Read the bounding box data from the bounding box kernel (see reference above)
p2bb = pd.read_csv(BB_DF).set_index("Image")

In [26]:
old_stderr = sys.stderr
sys.stderr = open('/dev/null' if platform.system() != 'Windows' else 'nul', 'w')

sys.stderr = old_stderr

In [27]:
img_shape = (384, 384, 1)  # The image shape used by the model
anisotropy = 2.15  # The horizontal compression ratio
crop_margin = 0.05  # The margin added around the bounding box to compensate for bounding box inaccuracy

In [28]:
def build_transform(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    """
    Build a transformation matrix with the specified characteristics.
    """
    rotation = np.deg2rad(rotation)
    shear = np.deg2rad(shear)
    rotation_matrix = np.array(
        [[np.cos(rotation), np.sin(rotation), 0], [-np.sin(rotation), np.cos(rotation), 0], [0, 0, 1]])
    shift_matrix = np.array([[1, 0, height_shift], [0, 1, width_shift], [0, 0, 1]])
    shear_matrix = np.array([[1, np.sin(shear), 0], [0, np.cos(shear), 0], [0, 0, 1]])
    zoom_matrix = np.array([[1.0 / height_zoom, 0, 0], [0, 1.0 / width_zoom, 0], [0, 0, 1]])
    shift_matrix = np.array([[1, 0, -height_shift], [0, 1, -width_shift], [0, 0, 1]])
    return np.dot(np.dot(rotation_matrix, shear_matrix), np.dot(zoom_matrix, shift_matrix))

In [29]:
def read_cropped_image(p, augment):
    """
    @param p : the name of the picture to read
    @param augment: True/False if data augmentation should be performed
    @return a numpy array with the transformed image
    """
    # If an image id was given, convert to filename
    if p in h2p:
        p = h2p[p]
    size_x, size_y = p2size[p]

    # Determine the region of the original image we want to capture based on the bounding box.
    row = p2bb.loc[p]
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    dx = x1 - x0
    dy = y1 - y0
    x0 -= dx * crop_margin
    x1 += dx * crop_margin + 1
    y0 -= dy * crop_margin
    y1 += dy * crop_margin + 1
    if x0 < 0:
        x0 = 0
    if x1 > size_x:
        x1 = size_x
    if y0 < 0:
        y0 = 0
    if y1 > size_y:
        y1 = size_y
    dx = x1 - x0
    dy = y1 - y0
    if dx > dy * anisotropy:
        dy = 0.5 * (dx / anisotropy - dy)
        y0 -= dy
        y1 += dy
    else:
        dx = 0.5 * (dy * anisotropy - dx)
        x0 -= dx
        x1 += dx

    # Generate the transformation matrix
    trans = np.array([[1, 0, -0.5 * img_shape[0]], [0, 1, -0.5 * img_shape[1]], [0, 0, 1]])
    trans = np.dot(np.array([[(y1 - y0) / img_shape[0], 0, 0], [0, (x1 - x0) / img_shape[1], 0], [0, 0, 1]]), trans)
    if augment:
        trans = np.dot(build_transform(
            random.uniform(-5, 5),
            random.uniform(-5, 5),
            random.uniform(0.8, 1.0),
            random.uniform(0.8, 1.0),
            random.uniform(-0.05 * (y1 - y0), 0.05 * (y1 - y0)),
            random.uniform(-0.05 * (x1 - x0), 0.05 * (x1 - x0))
        ), trans)
    trans = np.dot(np.array([[1, 0, 0.5 * (y1 + y0)], [0, 1, 0.5 * (x1 + x0)], [0, 0, 1]]), trans)

    # Read the image, transform to black and white and comvert to numpy array
    img = read_raw_image(p).convert('L')
    img = img_to_array(img)

    # Apply affine transformation
    matrix = trans[:2, :2]
    offset = trans[:2, 2]
    img = img.reshape(img.shape[:-1])
    img = affine_transform(img, matrix, offset, output_shape=img_shape[:-1], order=1, mode='constant',
                           cval=np.average(img))
    img = img.reshape(img_shape)

    # Normalize to zero mean and unit variance
    img -= np.mean(img, keepdims=True)
    img /= np.std(img, keepdims=True) + K.epsilon()
    return img

In [30]:
def read_for_training(p):
    """
    Read and preprocess an image with data augmentation (random transform).
    """
    return read_cropped_image(p, True)


def read_for_validation(p):
    """
    Read and preprocess an image without data augmentation (use for testing).
    """
    return read_cropped_image(p, False)


p = list(tagged.keys())[312]

In [31]:
def subblock(x, filter, **kwargs):
    x = BatchNormalization()(x)
    y = x
    y = Conv2D(filter, (1, 1), activation='relu', **kwargs)(y)  # Reduce the number of features to 'filter'
    y = BatchNormalization()(y)
    y = Conv2D(filter, (3, 3), activation='relu', **kwargs)(y)  # Extend the feature field
    y = BatchNormalization()(y)
    y = Conv2D(K.int_shape(x)[-1], (1, 1), **kwargs)(y)  # no activation # Restore the number of original features
    y = Add()([x, y])  # Add the bypass connection
    y = Activation('relu')(y)
    return y


def build_model(lr, l2, activation='sigmoid'):
    ##############
    # BRANCH MODEL
    ##############
    regul = regularizers.l2(l2)
    optim = Adam(lr=lr)
    kwargs = {'padding': 'same', 'kernel_regularizer': regul}

    inp = Input(shape=img_shape)  # 384x384x1
    x = Conv2D(64, (9, 9), strides=2, activation='relu', **kwargs)(inp)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 96x96x64
    for _ in range(2):
        x = BatchNormalization()(x)
        x = Conv2D(64, (3, 3), activation='relu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128, (1, 1), activation='relu', **kwargs)(x)  # 48x48x128
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256, (1, 1), activation='relu', **kwargs)(x)  # 24x24x256
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384, (1, 1), activation='relu', **kwargs)(x)  # 12x12x384
    for _ in range(4):
        x = subblock(x, 96, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512, (1, 1), activation='relu', **kwargs)(x)  # 6x6x512
    for _ in range(4):
        x = subblock(x, 128, **kwargs)

    x = GlobalMaxPooling2D()(x)  # 512
    branch_model = Model(inp, x)

    ############
    # HEAD MODEL
    ############
    mid = 32
    xa_inp = Input(shape=branch_model.output_shape[1:])
    xb_inp = Input(shape=branch_model.output_shape[1:])
    x1 = Lambda(lambda x: x[0] * x[1])([xa_inp, xb_inp])
    x2 = Lambda(lambda x: x[0] + x[1])([xa_inp, xb_inp])
    x3 = Lambda(lambda x: K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4 = Lambda(lambda x: K.square(x))(x3)
    x = Concatenate()([x1, x2, x3, x4])
    x = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x = Conv2D(mid, (4, 1), activation='relu', padding='valid')(x)
    x = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x = Flatten(name='flatten')(x)

    # Weighted sum implemented as a Dense layer.
    x = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')

    ########################
    # SIAMESE NEURAL NETWORK
    ########################
    # Complete model is constructed by calling the branch model on each input image,
    # and then the head model on the resulting 512-vectors.
    img_a = Input(shape=img_shape)
    img_b = Input(shape=img_shape)
    xa = branch_model(img_a)
    xb = branch_model(img_b)
    x = head_model([xa, xb])
    model = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['binary_crossentropy', 'acc'])
    return model, branch_model, head_model


model, branch_model, head_model = build_model(64e-5, 0)

In [32]:
h2ws = {}
new_whale = 'new_whale'
for p, w in tagged.items():
    if w != new_whale:  # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
for h, ws in h2ws.items():
    if len(ws) > 1:
        h2ws[h] = sorted(ws)

In [33]:
len(h2ws), list(h2ws.items())[:5]

(15696,
 [('d26698c3271c757c', ['w_f48451c']),
  ('ba8cc231ad489b77', ['w_c3d896a']),
  ('bbcad234a52d0f0b', ['w_20df2c5']),
  ('e91b8cc2f270723d', ['w_dd88965']),
  ('e99a96243f89711e', ['w_64404ac'])])

In [34]:
# For each whale, find the unambiguous images ids.
w2hs = {}
for h, ws in h2ws.items():
    if len(ws) == 1:  # Use only unambiguous pictures
        w = ws[0]
        if w not in w2hs: w2hs[w] = []
        if h not in w2hs[w]: w2hs[w].append(h)
for w, hs in w2hs.items():
    if len(hs) > 1:
        w2hs[w] = sorted(hs)

In [35]:
len(w2hs), list(w2hs.items())[:3]

(5004,
 [('w_f48451c',
   ['93436cbc5a879761',
    '96b1699e46397c92',
    '96d9ac9bc62dc1c2',
    'a2ddce3324c8bc0f',
    'afcbd0302fcd9033',
    'b08f8e73332cccc3',
    'b0cf8c3376ac511b',
    'b3e81cc92330decd',
    'b688cd3332e4cc9b',
    'ba9899aa9a597a26',
    'c1363f8d27b8986c',
    'c79a69c31e638c3c',
    'd0be5cb26790c83d',
    'd26698c3271c757c']),
  ('w_c3d896a',
   ['b33cced372343131',
    'ba8cc231ad489b77',
    'bacac4b53d484a73',
    'ead4952568da7634']),
  ('w_20df2c5',
   ['8fc0f53588fb2a4a',
    'b98ad332b529cc39',
    'bbcad234a52d0f0b',
    'fa17c07078391e97'])])

In [36]:
train = []  # A list of training image ids
for hs in w2hs.values():
    if len(hs) > 1:
        train += hs
random.shuffle(train)
train_set = set(train)

In [37]:
len(train_set), list(train_set)[:5]

(13623,
 ['df3881c2e4795f0a',
  'ea5a9525691a87e5',
  'eece81f0b234acac',
  'fa95807095699e4f',
  'bbc6a469993c4e32'])

In [38]:
w2ts = {}  # Associate the image ids from train to each whale id.
for w, hs in w2hs.items():
    for h in hs:
        if h in train_set:
            if w not in w2ts:
                w2ts[w] = []
            if h not in w2ts[w]:
                w2ts[w].append(h)
for w, ts in w2ts.items():
    w2ts[w] = np.array(ts)

In [39]:
len(w2ts), list(w2ts.items())[:5]

(2931,
 [('w_f48451c',
   array(['93436cbc5a879761', '96b1699e46397c92', '96d9ac9bc62dc1c2',
          'a2ddce3324c8bc0f', 'afcbd0302fcd9033', 'b08f8e73332cccc3',
          'b0cf8c3376ac511b', 'b3e81cc92330decd', 'b688cd3332e4cc9b',
          'ba9899aa9a597a26', 'c1363f8d27b8986c', 'c79a69c31e638c3c',
          'd0be5cb26790c83d', 'd26698c3271c757c'], dtype='<U16')),
  ('w_c3d896a',
   array(['b33cced372343131', 'ba8cc231ad489b77', 'bacac4b53d484a73',
          'ead4952568da7634'], dtype='<U16')),
  ('w_20df2c5',
   array(['8fc0f53588fb2a4a', 'b98ad332b529cc39', 'bbcad234a52d0f0b',
          'fa17c07078391e97'], dtype='<U16')),
  ('w_dd88965',
   array(['a86986d6b03b6fc4', 'a9c4956a562d8cbb', 'ad88d2b06f727c89',
          'bc4ed0f2a7e168a8', 'bcc19067bc28cb3e', 'bcc5c27249b48d7a',
          'bdc0d327259c68da', 'bdc0d427e0b8c9ba', 'da8582702dcb62bf',
          'e89d9662719c4e93', 'e8d985f13e25c887', 'e91b8cc2f270723d',
          'ea8781f2b47da80b', 'ea9986e2b0e81dec', 'ea9d8660b0699d6e'

In [40]:
t2i = {}  # The position in train of each training image id
for i, t in enumerate(train):
    t2i[t] = i

In [41]:
len(t2i), list(t2i.items())[:5]

(13623,
 [('eadb98a5e1781c0d', 0),
  ('eb9ec9f28520b525', 1),
  ('927a6d8d9b36244d', 2),
  ('f84ad3a1e232b4ad', 3),
  ('e8d885e3b2c3acd1', 4)])

In [42]:
class TrainingData(Sequence):
    def __init__(self, score, steps=1000, batch_size=32):
        """
        @param score the cost matrix for the picture matching
        @param steps the number of epoch we are planning with this score matrix
        """
        super(TrainingData, self).__init__()
        self.score = -score  # Maximizing the score is the same as minimizing -score.
        self.steps = steps
        self.batch_size = batch_size
        for ts in w2ts.values():
            idxs = [t2i[t] for t in ts]
            for i in idxs:
                for j in idxs:
                    self.score[
                        i, j] = 10000.0  # Set a large value for matching whales -- eliminates this potential pairing
        self.on_epoch_end()

    def __getitem__(self, index):
        start = self.batch_size * index
        end = min(start + self.batch_size, len(self.match) + len(self.unmatch))
        size = end - start
        assert size > 0
        a = np.zeros((size,) + img_shape, dtype=K.floatx())
        b = np.zeros((size,) + img_shape, dtype=K.floatx())
        c = np.zeros((size, 1), dtype=K.floatx())
        j = start // 2
        for i in range(0, size, 2):
            a[i, :, :, :] = read_for_training(self.match[j][0])
            b[i, :, :, :] = read_for_training(self.match[j][1])
            c[i, 0] = 1  # This is a match
            a[i + 1, :, :, :] = read_for_training(self.unmatch[j][0])
            b[i + 1, :, :, :] = read_for_training(self.unmatch[j][1])
            c[i + 1, 0] = 0  # Different whales
            j += 1
        return [a, b], c

    def on_epoch_end(self):
        if self.steps <= 0: return  # Skip this on the last epoch.
        self.steps -= 1
        self.match = []
        self.unmatch = []
        _, _, x = lapjv(self.score)  # Solve the linear assignment problem
        y = np.arange(len(x), dtype=np.int32)

        # Compute a derangement for matching whales
        for ts in w2ts.values():
            d = ts.copy()
            while True:
                random.shuffle(d)
                if not np.any(ts == d): break
            for ab in zip(ts, d): self.match.append(ab)

        # Construct unmatched whale pairs from the LAP solution.
        for i, j in zip(x, y):
            if i == j:
                print(self.score)
                print(x)
                print(y)
                print(i, j)
            assert i != j
            self.unmatch.append((train[i], train[j]))

        # Force a different choice for an eventual next epoch.
        self.score[x, y] = 10000.0
        self.score[y, x] = 10000.0
        random.shuffle(self.match)
        random.shuffle(self.unmatch)
        # print(len(self.match), len(train), len(self.unmatch), len(train))
        assert len(self.match) == len(train) and len(self.unmatch) == len(train)

    def __len__(self):
        return (len(self.match) + len(self.unmatch) + self.batch_size - 1) // self.batch_size

In [43]:
# Test on a batch of 32 with random costs.
score = np.random.random_sample(size=(len(train), len(train)))
data = TrainingData(score)
(a, b), c = data[0]

In [44]:
# A Keras generator to evaluate only the BRANCH MODEL
class FeatureGen(Sequence):
    def __init__(self, data, batch_size=64, verbose=1):
        super(FeatureGen, self).__init__()
        self.data = data
        self.batch_size = batch_size
        self.verbose = verbose
        if self.verbose > 0: self.progress = tqdm(total=len(self), desc='Features')

    def __getitem__(self, index):
        start = self.batch_size * index
        size = min(len(self.data) - start, self.batch_size)
        a = np.zeros((size,) + img_shape, dtype=K.floatx())
        for i in range(size): a[i, :, :, :] = read_for_validation(self.data[start + i])
        if self.verbose > 0:
            self.progress.update()
            if self.progress.n >= len(self): self.progress.close()
        return a

    def __len__(self):
        return (len(self.data) + self.batch_size - 1) // self.batch_size


class ScoreGen(Sequence):
    def __init__(self, x, y=None, batch_size=2048, verbose=1):
        super(ScoreGen, self).__init__()
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.verbose = verbose
        if y is None:
            self.y = self.x
            self.ix, self.iy = np.triu_indices(x.shape[0], 1)
        else:
            self.iy, self.ix = np.indices((y.shape[0], x.shape[0]))
            self.ix = self.ix.reshape((self.ix.size,))
            self.iy = self.iy.reshape((self.iy.size,))
        self.subbatch = (len(self.x) + self.batch_size - 1) // self.batch_size
        if self.verbose > 0:
            self.progress = tqdm(total=len(self), desc='Scores')

    def __getitem__(self, index):
        start = index * self.batch_size
        end = min(start + self.batch_size, len(self.ix))
        a = self.y[self.iy[start:end], :]
        b = self.x[self.ix[start:end], :]
        if self.verbose > 0:
            self.progress.update()
            if self.progress.n >= len(self): self.progress.close()
        return [a, b]

    def __len__(self):
        return (len(self.ix) + self.batch_size - 1) // self.batch_size

In [45]:
def set_lr(model, lr):
    K.set_value(model.optimizer.lr, float(lr))


def get_lr(model):
    return K.get_value(model.optimizer.lr)


def score_reshape(score, x, y=None):
    """
    Tranformed the packed matrix 'score' into a square matrix.
    @param score the packed matrix
    @param x the first image feature tensor
    @param y the second image feature tensor if different from x
    @result the square matrix
    """
    if y is None:
        # When y is None, score is a packed upper triangular matrix.
        # Unpack, and transpose to form the symmetrical lower triangular matrix.
        m = np.zeros((x.shape[0], x.shape[0]), dtype=K.floatx())
        m[np.triu_indices(x.shape[0], 1)] = score.squeeze()
        m += m.transpose()
    else:
        m = np.zeros((y.shape[0], x.shape[0]), dtype=K.floatx())
        iy, ix = np.indices((y.shape[0], x.shape[0]))
        ix = ix.reshape((ix.size,))
        iy = iy.reshape((iy.size,))
        m[iy, ix] = score.squeeze()
    return m


def compute_score(verbose=1):
    """
    Compute the score matrix by scoring every pictures from the training set against every other picture O(n^2).
    """
    features = branch_model.predict_generator(FeatureGen(train, verbose=verbose), max_queue_size=12, workers=16,
                                              verbose=0)
    score = head_model.predict_generator(ScoreGen(features, verbose=verbose), max_queue_size=12, workers=16, verbose=0)
    score = score_reshape(score, features)
    return features, score


def make_steps(step, ampl):
    """
    Perform training epochs
    @param step Number of epochs to perform
    @param ampl the K, the randomized component of the score matrix.
    """
    global w2ts, t2i, steps, features, score, histories

    # shuffle the training pictures
    random.shuffle(train)

    # Map whale id to the list of associated training picture hash value
    w2ts = {}
    for w, hs in w2hs.items():
        for h in hs:
            if h in train_set:
                if w not in w2ts: w2ts[w] = []
                if h not in w2ts[w]: w2ts[w].append(h)
    for w, ts in w2ts.items(): w2ts[w] = np.array(ts)

    # Map training picture hash value to index in 'train' array    
    t2i = {}
    for i, t in enumerate(train): t2i[t] = i

    # Compute the match score for each picture pair
    features, score = compute_score()

    # Train the model for 'step' epochs
    history = model.fit_generator(
        TrainingData(score + ampl * np.random.random_sample(size=score.shape), steps=step, batch_size=32),
        initial_epoch=steps, epochs=steps + step, max_queue_size=12, workers=16, verbose=1).history
    steps += step

    # Collect history data
    history['epochs'] = steps
    history['ms'] = np.mean(score)
    history['lr'] = get_lr(model)
    print(history['epochs'], history['lr'], history['ms'])
    histories.append(history)

In [46]:
def prepare_submission(threshold, filename):
    """
    Generate a Kaggle submission file.
    @param threshold the score given to 'new_whale'
    @param filename the submission file name
    """
    vtop = 0
    vhigh = 0
    pos = [0, 0, 0, 0, 0, 0]
    with open(filename, 'wt', newline='\n') as f:
        f.write('Image,Id\n')
        for i, p in enumerate(tqdm(submit)):
            t = []
            s = set()
            a = score[i, :]
            for j in list(reversed(np.argsort(a))):
                h = known[j]
                if a[j] < threshold and new_whale not in s:
                    pos[len(t)] += 1
                    s.add(new_whale)
                    t.append(new_whale)
                    if len(t) == 5: break;
                for w in h2ws[h]:
                    assert w != new_whale
                    if w not in s:
                        if a[j] > 1.0:
                            vtop += 1
                        elif a[j] >= threshold:
                            vhigh += 1
                        s.add(w)
                        t.append(w)
                        if len(t) == 5: break;
                if len(t) == 5: break;
            if new_whale not in s: pos[5] += 1
            assert len(t) == 5 and len(s) == 5
            f.write(p + ',' + ' '.join(t[:5]) + '\n')
    return vtop, vhigh, pos

In [48]:
histories = []
steps = 0

#############################
# Load standard model and compute score 
tmp = keras.models.load_model('standard-10.model')
model.set_weights(tmp.get_weights())

# Find elements from training sets not 'new_whale'
tic = time.time()
h2ws = {}
for p, w in tagged.items():
    if w != new_whale:  # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
known = sorted(list(h2ws.keys()))

# Dictionary of picture indices
h2i = {}
for i, h in enumerate(known): h2i[h] = i

# Evaluate the model.
fknown1 = branch_model.predict_generator(FeatureGen(known), max_queue_size=20, workers=16, verbose=1)
fsubmit1 = branch_model.predict_generator(FeatureGen(submit), max_queue_size=20, workers=16, verbose=1)
score1 = head_model.predict_generator(ScoreGen(fknown1, fsubmit1), max_queue_size=20, workers=16, verbose=1)
score1 = score_reshape(score1, fknown1, fsubmit1)

###########################
# Load bootstrap model and compute score 
tmp = keras.models.load_model('../data/models/mpiotte-bootstrap.model')
model.set_weights(tmp.get_weights())

# Find elements from training sets not 'new_whale'
tic = time.time()
h2ws = {}
for p, w in tagged.items():
    if w != new_whale:  # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
known = sorted(list(h2ws.keys()))

# Dictionary of picture indices
h2i = {}
for i, h in enumerate(known): h2i[h] = i

# Evaluate the model.
fknown2 = branch_model.predict_generator(FeatureGen(known), max_queue_size=20, workers=16, verbose=1)
fsubmit2 = branch_model.predict_generator(FeatureGen(submit), max_queue_size=20, workers=16, verbose=1)
score2 = head_model.predict_generator(ScoreGen(fknown2, fsubmit2), max_queue_size=20, workers=16, verbose=1)
score2 = score_reshape(score2, fknown2, fsubmit2)

246/246 [==============================] - 171s 694ms/step


125/125 [==============================] - 93s 745ms/step


61006/61006 [==============================] - 533s 9ms/step


246/246 [==============================] - 166s 673ms/step


125/125 [==============================] - 83s 667ms/step


61006/61006 [==============================] - 579s 9ms/step


In [ ]:
# epoch -> 10
make_steps(10, 1000)
ampl = 100.0
for _ in range(2):
    print('noise ampl.  = ', ampl)
    make_steps(5, ampl)
    ampl = max(1.0, 100 ** -0.1 * ampl)
model.save('standard-10.model')


Epoch 1/10
852/852 [==============================] - 741s 870ms/step - loss: 0.2061 - binary_crossentropy: 0.2061 - acc: 0.9213
Epoch 2/10
852/852 [==============================] - 688s 808ms/step - loss: 0.1505 - binary_crossentropy: 0.1505 - acc: 0.9400
Epoch 3/10
852/852 [==============================] - 737s 865ms/step - loss: 0.1491 - binary_crossentropy: 0.1491 - acc: 0.9415
Epoch 4/10
852/852 [==============================] - 690s 809ms/step - loss: 0.1479 - binary_crossentropy: 0.1479 - acc: 0.9406
Epoch 5/10
852/852 [==============================] - 696s 817ms/step - loss: 0.1405 - binary_crossentropy: 0.1405 - acc: 0.9452
Epoch 6/10
852/852 [==============================] - 702s 823ms/step - loss: 0.1414 - binary_crossentropy: 0.1414 - acc: 0.9435
Epoch 7/10
852/852 [==============================] - 725s 851ms/step - loss: 0.1363 - binary_crossentropy: 0.1363 - acc: 0.9461
Epoch 8/10
852/852 [==============================] - 642s 753ms/step - loss: 0.1331 - binary_cr


Epoch 11/15
852/852 [==============================] - 603s 708ms/step - loss: 0.3073 - binary_crossentropy: 0.3073 - acc: 0.8670
Epoch 12/15
852/852 [==============================] - 600s 704ms/step - loss: 0.2450 - binary_crossentropy: 0.2450 - acc: 0.8968
Epoch 13/15
852/852 [==============================] - 630s 739ms/step - loss: 0.2072 - binary_crossentropy: 0.2072 - acc: 0.9136
Epoch 14/15
852/852 [==============================] - 639s 750ms/step - loss: 0.1888 - binary_crossentropy: 0.1888 - acc: 0.9223
Epoch 15/15
852/852 [==============================] - 552s 648ms/step - loss: 0.1699 - binary_crossentropy: 0.1699 - acc: 0.9312
15 0.00064 0.085166946
noise ampl.  =  63.09573444801933



Epoch 16/20
852/852 [==============================] - 678s 795ms/step - loss: 0.2658 - binary_crossentropy: 0.2658 - acc: 0.8846
Epoch 17/20
852/852 [==============================] - 741s 869ms/step - loss: 0.2096 - binary_crossentropy: 0.2096 - acc: 0.9116
Epoch 18/20
852/852 [==============================] - 835s 980ms/step - loss: 0.1827 - binary_crossentropy: 0.1827 - acc: 0.9245
Epoch 19/20
852/852 [==============================] - 910s 1s/step - loss: 0.1659 - binary_crossentropy: 0.1659 - acc: 0.9312
Epoch 20/20
852/852 [==============================] - 549s 645ms/step - loss: 0.1494 - binary_crossentropy: 0.1494 - acc: 0.9378
20 0.00064 0.03765761


In [ ]:
# epoch -> 150
for _ in range(18): make_steps(5, 1.0)
model.save('standard-150.model')


Epoch 21/25
852/852 [==============================] - 595s 699ms/step - loss: 0.3357 - binary_crossentropy: 0.3357 - acc: 0.8552
Epoch 22/25
852/852 [==============================] - 597s 701ms/step - loss: 0.2739 - binary_crossentropy: 0.2739 - acc: 0.8838
Epoch 23/25
852/852 [==============================] - 603s 708ms/step - loss: 0.2486 - binary_crossentropy: 0.2486 - acc: 0.8977
Epoch 24/25
852/852 [==============================] - 606s 711ms/step - loss: 0.2347 - binary_crossentropy: 0.2347 - acc: 0.9043
Epoch 25/25
852/852 [==============================] - 551s 646ms/step - loss: 0.2186 - binary_crossentropy: 0.2186 - acc: 0.9105
25 0.00064 0.025107086



Epoch 26/30
852/852 [==============================] - 659s 774ms/step - loss: 0.2791 - binary_crossentropy: 0.2791 - acc: 0.8833
Epoch 27/30
852/852 [==============================] - 695s 816ms/step - loss: 0.2210 - binary_crossentropy: 0.2210 - acc: 0.9112
Epoch 28/30
852/852 [==============================] - 757s 889ms/step - loss: 0.2062 - binary_crossentropy: 0.2062 - acc: 0.9175
Epoch 29/30
852/852 [==============================] - 829s 973ms/step - loss: 0.1896 - binary_crossentropy: 0.1896 - acc: 0.9247
Epoch 30/30
852/852 [==============================] - 549s 644ms/step - loss: 0.1811 - binary_crossentropy: 0.1811 - acc: 0.9281
30 0.00064 0.026784025



Epoch 31/35
852/852 [==============================] - 642s 753ms/step - loss: 0.2539 - binary_crossentropy: 0.2539 - acc: 0.8964
Epoch 32/35
852/852 [==============================] - 680s 798ms/step - loss: 0.2055 - binary_crossentropy: 0.2055 - acc: 0.9200
Epoch 33/35
852/852 [==============================] - 720s 845ms/step - loss: 0.1857 - binary_crossentropy: 0.1857 - acc: 0.9265
Epoch 34/35
852/852 [==============================] - 777s 912ms/step - loss: 0.1707 - binary_crossentropy: 0.1707 - acc: 0.9327
Epoch 35/35
852/852 [==============================] - 554s 650ms/step - loss: 0.1619 - binary_crossentropy: 0.1619 - acc: 0.9370
35 0.00064 0.021102527



Epoch 36/40
852/852 [==============================] - 687s 807ms/step - loss: 0.2338 - binary_crossentropy: 0.2338 - acc: 0.9056
Epoch 37/40
852/852 [==============================] - 765s 898ms/step - loss: 0.1848 - binary_crossentropy: 0.1848 - acc: 0.9287
Epoch 38/40
852/852 [==============================] - 843s 990ms/step - loss: 0.1647 - binary_crossentropy: 0.1647 - acc: 0.9354
Epoch 39/40
852/852 [==============================] - 944s 1s/step - loss: 0.1512 - binary_crossentropy: 0.1512 - acc: 0.9425
Epoch 40/40
852/852 [==============================] - 550s 646ms/step - loss: 0.1464 - binary_crossentropy: 0.1464 - acc: 0.9418
40 0.00064 0.024891052



Epoch 41/45
852/852 [==============================] - 785s 922ms/step - loss: 0.2274 - binary_crossentropy: 0.2274 - acc: 0.9101
Epoch 42/45
852/852 [==============================] - 917s 1s/step - loss: 0.1850 - binary_crossentropy: 0.1850 - acc: 0.9263
Epoch 43/45
852/852 [==============================] - 1062s 1s/step - loss: 0.1627 - binary_crossentropy: 0.1627 - acc: 0.9379
Epoch 44/45
852/852 [==============================] - 1190s 1s/step - loss: 0.1526 - binary_crossentropy: 0.1526 - acc: 0.9401
Epoch 45/45
852/852 [==============================] - 551s 647ms/step - loss: 0.1397 - binary_crossentropy: 0.1397 - acc: 0.9447
45 0.00064 0.018267082



Epoch 46/50
852/852 [==============================] - 829s 974ms/step - loss: 0.2167 - binary_crossentropy: 0.2167 - acc: 0.9121
Epoch 47/50
852/852 [==============================] - 938s 1s/step - loss: 0.1694 - binary_crossentropy: 0.1694 - acc: 0.9350
Epoch 48/50
852/852 [==============================] - 1046s 1s/step - loss: 0.1571 - binary_crossentropy: 0.1571 - acc: 0.9379
Epoch 49/50
852/852 [==============================] - 1288s 2s/step - loss: 0.1376 - binary_crossentropy: 0.1376 - acc: 0.9465
Epoch 50/50
852/852 [==============================] - 550s 645ms/step - loss: 0.1318 - binary_crossentropy: 0.1318 - acc: 0.9485
50 0.00064 0.021822374



Epoch 51/55
852/852 [==============================] - 907s 1s/step - loss: 0.2015 - binary_crossentropy: 0.2015 - acc: 0.9172
Epoch 52/55
852/852 [==============================] - 1024s 1s/step - loss: 0.1635 - binary_crossentropy: 0.1635 - acc: 0.9348
Epoch 53/55
852/852 [==============================] - 1205s 1s/step - loss: 0.1461 - binary_crossentropy: 0.1461 - acc: 0.9439
Epoch 54/55
852/852 [==============================] - 1400s 2s/step - loss: 0.1374 - binary_crossentropy: 0.1374 - acc: 0.9464
Epoch 55/55
852/852 [==============================] - 553s 649ms/step - loss: 0.1299 - binary_crossentropy: 0.1299 - acc: 0.9493
55 0.00064 0.012899656



Epoch 56/60
852/852 [==============================] - 1212s 1s/step - loss: 0.2004 - binary_crossentropy: 0.2004 - acc: 0.9202
Epoch 57/60
852/852 [==============================] - 1522s 2s/step - loss: 0.1526 - binary_crossentropy: 0.1526 - acc: 0.9404
Epoch 58/60
852/852 [==============================] - 1812s 2s/step - loss: 0.1380 - binary_crossentropy: 0.1380 - acc: 0.9470
Epoch 59/60
851/852 [============================>.] - ETA: 0s - loss: 0.1341 - binary_crossentropy: 0.1341 - acc: 0.9474

In [ ]:
# epoch -> 200
set_lr(model, 16e-5)
for _ in range(10): make_steps(5, 0.5)
model.save('standard-200.model')

In [ ]:
# epoch -> 240
set_lr(model, 4e-5)
for _ in range(8): make_steps(5, 0.25)
model.save('standard-240.model')

In [ ]:
# epoch -> 250
set_lr(model, 1e-5)
for _ in range(2): make_steps(5, 0.25)
model.save('standard-250.model')

In [ ]:
# epoch -> 300
weights = model.get_weights()
model, branch_model, head_model = build_model(64e-5, 0.0002)
model.set_weights(weights)
for _ in range(10): make_steps(5, 1.0)
model.save('standard-300.model')

In [ ]:
# epoch -> 350
set_lr(model, 16e-5)
for _ in range(10): make_steps(5, 0.5)
model.save('standard-350.model')

In [ ]:
# epoch -> 390
set_lr(model, 4e-5)
for _ in range(8): make_steps(5, 0.25)
model.save('standard-390.model')

In [ ]:
# epoch -> 400
set_lr(model, 1e-5)
for _ in range(2): make_steps(5, 0.25)
model.save('standard-400.model')

In [49]:
# Do the weighthing exaclty as Martin suggests
score = 0.45*score1 + 0.55*score2

In [50]:
# Generate the subsmission file.
prepare_submission(0.92, '19_submission_0.45_standard_0.55_boostrap.csv')
toc = time.time()
print("Submission time: ", (toc - tic) / 60.)


Submission time:  19.918695465723673
